In [1]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


In [2]:
from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


In [3]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Keras modules

In [11]:
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers.core import Dense, Flatten, Activation
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D, Input,Add

## Cargo datos

In [12]:
from keras.datasets import cifar10
((x, y), (x_test,y_test)) = cifar10.load_data()

In [13]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1)

## Paso a float

In [14]:
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

In [15]:
y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_valid)

## Red neuronal: CNN tipo Inception

In [75]:
net_input = Input(shape=(32,32,3))

def inception_block(input_layer,filters=64):
    tower_1 = Conv2D(filters, (1,1), padding='same', activation='relu')(input_layer)
    tower_1 = Conv2D(filters, (3,3), padding='same', activation='relu')(tower_1)
    tower_2 = Conv2D(filters, (1,1), padding='same', activation='relu')(input_layer)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_layer)
    tower_3 = Conv2D(filters, (1,1), padding='same', activation='relu')(tower_3)
    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output


net=inception_block(net_input,filters=4)
net=inception_block(net,filters=4)
net=inception_block(net,filters=4)
net=inception_block(net,filters=4)
net=inception_block(net,filters=4)
net=inception_block(net,filters=4)

net=Flatten()(net)

net=Dense(units=128, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=256, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=10, activation = 'softmax')(net)

model=Model(net_input,net)

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_321 (Conv2D)             (None, 32, 32, 4)    16          input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_323 (Conv2D)             (None, 32, 32, 4)    16          input_16[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_65 (MaxPooling2D) (None, 32, 32, 3)    0           input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_322

In [76]:
# Callbacks
checkpoint_file='checkpoint.cifar10-cnn-tipo-Inception.hdf5'
earlystop = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=2)
checkpointer = ModelCheckpoint(monitor="val_acc",filepath=checkpoint_file, verbose=1, save_best_only=True)

In [77]:
x_train.shape

(45000, 32, 32, 3)

In [78]:
y_train_categorical.shape

(45000, 10)

In [79]:
batch_size = 64
epocs = 100

history = model.fit(x_train, 
        y_train_categorical,
        epochs=epocs, batch_size=batch_size, 
        verbose=1, 
        validation_data = (x_valid, y_val_categorical),
        callbacks=[checkpointer, earlystop],
    )

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 24s 534us/step - loss: 1.7623 - acc: 0.3450 - val_loss: 1.4632 - val_acc: 0.4676

Epoch 00001: val_acc improved from -inf to 0.46760, saving model to checkpoint.mnist-cnn-tipo-Inception.hdf5
Epoch 2/100
45000/45000 [==============================] - 20s 451us/step - loss: 1.3716 - acc: 0.5031 - val_loss: 1.2513 - val_acc: 0.5506

Epoch 00002: val_acc improved from 0.46760 to 0.55060, saving model to checkpoint.mnist-cnn-tipo-Inception.hdf5
Epoch 3/100
45000/45000 [==============================] - 20s 451us/step - loss: 1.1941 - acc: 0.5700 - val_loss: 1.1635 - val_acc: 0.5810

Epoch 00003: val_acc improved from 0.55060 to 0.58100, saving model to checkpoint.mnist-cnn-tipo-Inception.hdf5
Epoch 4/100
45000/45000 [==============================] - 20s 452us/step - loss: 1.0460 - acc: 0.6241 - val_loss: 1.1467 - val_acc: 0.5948

Epoch 00004: val_acc improved from 0.58100 to 0.59480,

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(18,5))
ax1.plot(history.history['loss'], label="loss")
ax1.plot(history.history['val_loss'], label="val_loss")
ax1.legend()

ax2.plot(history.history['acc'], label="acc")
ax2.plot(history.history['val_acc'], label="val_acc")
ax2.legend()
plt.show();